In [1]:
!pip install -r requirements.txt

# Parte 1 – Introducción al Tracking

In [2]:
import mlflow
import statistics
import random

In [3]:
mlflow.set_experiment("mlflow_basico")

def modelo_promedio(valores):
    return statistics.mean(valores)

with mlflow.start_run(run_name="ejemplo_promedio"):

    lista_valores = [random.randint(1, 100) for _ in range(10)]
    mlflow.log_param("tamaño_lista", len(lista_valores))
    mlflow.log_param("valores", lista_valores)

    resultado = modelo_promedio(lista_valores)
    mlflow.log_metric("promedio", resultado)

    with open("resultado.txt", "w") as f:
        f.write(f"Valores: {lista_valores}\n")
        f.write(f"Promedio: {resultado}\n")

    mlflow.log_artifact("resultado.txt")

# Parte 2 – Registro de Modelos de Machine Learning

In [4]:
import mlflow
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score, f1_score, precision_score
import logging
from mlflow.tracking import MlflowClient

In [5]:
logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)

mlflow.set_experiment("breast_cancer_experiment")

with mlflow.start_run(run_name = "breast_cancer_logistic_regression_v1") as run:

    X, y = load_breast_cancer(return_X_y= True)
    params = {"C": 0.1, "random_state": 42, "max_iter": 1000, "solver": "liblinear", "penalty": "l2"}
    mlflow.log_params(params)
    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))
    mlflow.log_metric("f1_score", f1_score(y, y_pred))
    mlflow.log_metric("precision", precision_score(y, y_pred))
    mlflow.sklearn.log_model(lr, "breast_cancer_logistic_regression_model")
    result = mlflow.register_model(
        model_uri=f"runs:/{run.info.run_id}/breast_cancer_logistic_regression_model",
        name="bc_logistic_regression_model"
    )

2025/11/10 18:38:35 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/10 18:38:40 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'bc_logistic_regression_model' already exists. Creating a new version of this model...
2025/11/10 18:38:40 WARNING mlflow.tracking._model_registry.fluent: Run with id fc47bebca63848e2a482c3c805c51f01 has no artifacts at artifact path 'breast_cancer_logistic_regression_model', registering model based on models:/m-03f0a7fd8c734523875cdfeac432388a instead
Created version '2' of model 'bc_logistic_regression_model'.


In [6]:
# Promover a Staging

client = MlflowClient()
model_name = "bc_logistic_regression_model"
version = 1

client.transition_model_version_stage(
    name=model_name,
    version=version,
    stage="Staging"
)

C:\Users\Patecnologicos\AppData\Local\Temp\ipykernel_30280\4097694002.py:7: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1762817435464, current_stage='Staging', deployment_job_state=None, description=None, last_updated_timestamp=1762817920376, metrics=[<Metric: dataset_digest=None, dataset_name=None, key='accuracy', model_id='m-744cd667c32c4318a84eeab70d6d6408', run_id='1519f110a4c641359237ce009fa9dbd4', step=0, timestamp=1762817428533, value=0.9472759226713533>,
 <Metric: dataset_digest=None, dataset_name=None, key='f1_score', model_id='m-744cd667c32c4318a84eeab70d6d6408', run_id='1519f110a4c641359237ce009fa9dbd4', step=0, timestamp=1762817428539, value=0.9584487534626038>,
 <Metric: dataset_digest=None, dataset_name=None, key='precision', model_id='m-744cd667c32c4318a84eeab70d6d6408', run_id='1519f110a4c641359237ce009fa9dbd4', step=0, timestamp=1762817428543, value=0.947945205479452>], model_id='m-744cd667c32c4318a84eeab70d6d6408', name='bc_logistic_regression_model', params={'C': '0.1',
 'max_iter': '1000',
 'penalty': 'l2',
 'random_state': '42',
 'solver

In [7]:
# Promover a Production

client.transition_model_version_stage(
    name=model_name,
    version=version,
    stage="Production"
)

C:\Users\Patecnologicos\AppData\Local\Temp\ipykernel_30280\1457271585.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1762817435464, current_stage='Production', deployment_job_state=None, description=None, last_updated_timestamp=1762817920417, metrics=[<Metric: dataset_digest=None, dataset_name=None, key='accuracy', model_id='m-744cd667c32c4318a84eeab70d6d6408', run_id='1519f110a4c641359237ce009fa9dbd4', step=0, timestamp=1762817428533, value=0.9472759226713533>,
 <Metric: dataset_digest=None, dataset_name=None, key='f1_score', model_id='m-744cd667c32c4318a84eeab70d6d6408', run_id='1519f110a4c641359237ce009fa9dbd4', step=0, timestamp=1762817428539, value=0.9584487534626038>,
 <Metric: dataset_digest=None, dataset_name=None, key='precision', model_id='m-744cd667c32c4318a84eeab70d6d6408', run_id='1519f110a4c641359237ce009fa9dbd4', step=0, timestamp=1762817428543, value=0.947945205479452>], model_id='m-744cd667c32c4318a84eeab70d6d6408', name='bc_logistic_regression_model', params={'C': '0.1',
 'max_iter': '1000',
 'penalty': 'l2',
 'random_state': '42',
 'sol

# Parte 3 – Tracking de Modelos de Lenguaje (LLMs)

In [8]:
import os
import time
import mlflow
from openai import OpenAI
from dotenv import load_dotenv
from mlflow.tracking import MlflowClient

load_dotenv()
gemini_key = os.environ.get("GEMINI_API_KEY")

gemini = OpenAI(base_url="https://generativelanguage.googleapis.com/v1beta/openai/", api_key=gemini_key)
ollama = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

# Prompt de prueba
prompt = "¿Que es la IA?"

# Simulación de costo por token
def estimar_costo(tokens):
    return tokens * 0.0001

def ejecutar_llm(nombre_modelo, cliente, proveedor, temperatura, tipo_tarea, registry_name):
    mlflow.set_experiment("comparacion_llm_chat")

    with mlflow.start_run(run_name=f"{registry_name}_run") as run:
        inicio = time.time()
        response = cliente.chat.completions.create(
            model=nombre_modelo,
            messages=[{"role": "user", "content": prompt}],
            temperature=temperatura
        )
        fin = time.time()

        texto = response.choices[0].message.content
        tokens = response.usage.total_tokens
        latencia = fin - inicio
        costo = estimar_costo(tokens)

        # Log de parámetros
        mlflow.log_param("modelo", nombre_modelo)
        mlflow.log_param("temperatura", temperatura)
        mlflow.log_param("proveedor", proveedor)
        mlflow.log_param("tipo_tarea", tipo_tarea)

        # Log de métricas
        mlflow.log_metric("latencia", latencia)
        mlflow.log_metric("tokens", tokens)
        mlflow.log_metric("costo_estimado", costo)

        # Artefactos
        with open("prompt.txt", "w") as f:
            f.write(prompt)
        with open("respuesta.txt", "w") as f:
            f.write(texto)

        mlflow.log_artifact("prompt.txt")
        mlflow.log_artifact("respuesta.txt")

        # Registrar modelo en el Model Registry
        mlflow.register_model(
            model_uri=f"mlruns:/{run.info.run_id}/respuesta.txt",
            name=registry_name
        )

# Ejecutar ambos modelos
ejecutar_llm("gemini-2.5-flash-lite", gemini, "Gemini", 0.7, "chat", "llm_gemini_chat")
ejecutar_llm("llama3.2:3b", ollama, "Ollama", 0.7, "chat", "llm_ollama_chat")


Registered model 'llm_gemini_chat' already exists. Creating a new version of this model...
Created version '2' of model 'llm_gemini_chat'.
Registered model 'llm_ollama_chat' already exists. Creating a new version of this model...
Created version '2' of model 'llm_ollama_chat'.


In [9]:
# Promocion de los modelos a Staging

client = MlflowClient()

client.transition_model_version_stage(
    name="llm_gemini_chat",
    version=1,
    stage="Staging"
)

client.transition_model_version_stage(
    name="llm_ollama_chat",
    version=1,
    stage="Staging"
)


C:\Users\Patecnologicos\AppData\Local\Temp\ipykernel_30280\1122991501.py:5: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(
C:\Users\Patecnologicos\AppData\Local\Temp\ipykernel_30280\1122991501.py:11: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1762817584264, current_stage='Staging', deployment_job_state=None, description=None, last_updated_timestamp=1762817932971, metrics=None, model_id=None, name='llm_ollama_chat', params=None, run_id=None, run_link=None, source='mlruns:/446bc361e2ec434bb17a689a76f4da57/respuesta.txt', status='READY', status_message=None, tags={}, user_id=None, version=1>